############# Markdown note ##################

<div class="alert alert-block alert-info"> <b>NOTE</b> Use blue boxes for Tips and notes. </div>

<div class="alert alert-block alert-success"> Use green boxes sparingly, and only for some specific purpose that the other boxes can't cover. For example, if you have a lot of related content to link to, maybe you decide to use green boxes for related links from each section of a notebook. </div>

<div class="alert alert-block alert-warning"> Use yellow boxes for examples that are not inside code cells, or use for mathematical formulas if needed. </div>

<div class="alert alert-block alert-danger"> In general, just avoid the red boxes. </div>

<img src="<path>" width=20% style="margin-left:auto; margin-right:auto">

In [1]:
%%sh

# reset all programs
rm -rf debug*# MPI Introduction

An introduction to basic concept of **Message Passing Interface** (MPI)

# MPI Communications

Communications with **Message Passing Interface** (MPI)

## Point-to-point communications

Basic communication method provided by MPI library - communication between 2 processes.

* Source process `A` sends a **message** to destination process `B`, `B` then receives the message from `A`;
* Communication take places within a **communicator**;
* The processes are identified by their **rank** in the communicator.

<img src="./Images/send.png" width=40% style="margin-left:auto; margin-right:auto">

### Send and Recv

Calls used to send and receive a simple message.

* `MPI_Send`: see https://www-lb.open-mpi.org/doc/v4.1/man3/MPI_Send.3.php
* `MPI_Recv`: see https://www-lb.open-mpi.org/doc/v4.1/man3/MPI_Recv.3.php

In [37]:
%%writefile main_send_recv.cpp

#include <iostream>
#include <mpi.h>

int main(int argc, char **argv) 
{
    MPI_Init(&argc, &argv);
    
    int process_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &process_rank);
    
    int tag = 10;
    float a[2] = { 1.2, 3.4 };
    float b[2] = { 0.0, 0.0 } ;
    MPI_Status status;
    
    if (process_rank == 0)
        MPI_Send(&a, 2, MPI_FLOAT, 1, tag, MPI_COMM_WORLD);
    else if (process_rank == 1)
        MPI_Recv(&b, 2, MPI_FLOAT, 0, tag, MPI_COMM_WORLD, &status);

    std::cout<< "Process "<< process_rank<< " ";
    std::cout<< "a "<< a[0]<< ", "<< a[1]<< " ";
    std::cout<< "b "<< b[0]<< ", "<< b[1]<< std::endl; 
    std::cout<< "Status SOURCE: "<< status.MPI_SOURCE<< " ";
    std::cout<< "Status TAG "<< status.MPI_TAG<< std::endl;
    
    MPI_Finalize();
    return 0;
}

Overwriting main_send_recv.cpp


In [38]:
%%sh

# compile program
mkdir -p ./debug_send_recv
cd debug_send_recv
cmake -DSOURCES="main_send_recv.cpp" ..
make

-- Configuring done
-- Generating done
-- Build files have been written to: /home/geoscore/Desktop/GEO++/Courses/PhdMPI/2_MPI_Communications/debug_send_recv
Consolidate compiler generated dependencies of target 2_MPI_Communications
[ 50%] Building CXX object CMakeFiles/2_MPI_Communications.dir/main_send_recv.cpp.o
[100%] Linking CXX executable 2_MPI_Communications
[100%] Built target 2_MPI_Communications


In [39]:
%%sh

# run program
cd debug_send_recv
mpirun -np 2 2_MPI_Communications

Process 0 a 1.2, 3.4 b 0, 0
Status SOURCE: -438324504 Status TAG 32513
Process 1 a 1.2, 3.4 b 1.2, 3.4
Status SOURCE: 0 Status TAG 10


<div class="alert alert-block alert-warning"> <b>NOTE</b>: the use of the IF statements - remember each task runs exactly the same program. </div>

## Message

Composed by a **buffer** and an **envelope**.
 
* Data is exchanged in the buffer, an array of count elements of some particular **MPI data type**;
* The envelope identifies the message. A message could be exchanged **only if** the sender and receiver specify the correct envelope.

<img src="./Images/message.png" width=60% style="margin-left:auto; margin-right:auto">

## DataTypes

MPI Data types can be:
* Basic types
* Derived types (`MPI_Type_xxx` functions)

<div class="alert alert-block alert-info"> <b>NOTE</b>: a derived type can be built up from basic types. </div>

MPI defines **handles** to allow programmers to refer to data types and structures

<div class="alert alert-block alert-warning"> C/C++ handles are macros to structs (<code>#define MPI_INT</code> …) </div>

### C/C++ MPI Data Types

<img src="./Images/types.png" width=90% style="margin-left:auto; margin-right:auto">

DataTypes can be created with different MPI routines, for example:

* `MPI_Pack`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Pack.3.php
* `MPI_Type_create_struct`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_create_struct.3.php
* ...

<div class="alert alert-block alert-success"> Before using a new DataType, we shall <b>commit</b> it. </div>

* `MPI_Type_commit`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_commit.3.php

<div class="alert alert-block alert-danger"> <b>REMARK</b>: Once a new data type is created we shall <b>destroy</b> it before closing the application:</div>

* `MPI_Type_free`: https://www.open-mpi.org/doc/v4.1/man3/MPI_Type_free.3.php

<div class="alert alert-block alert-info"> <b>NOTE</b>: We are going to see more about data types when we are going to see <b>advanced</b> data types. </div>

In [1]:
%%writefile main_struct.cpp

#include <iostream>
#include <mpi.h>

struct Car 
{
    int Model;
    int Color;
};

int main(int argc, char **argv) 
{
    MPI_Init(&argc, &argv);

    const int tag = 13;
    int size, rank;
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);

    // create a MPI type for struct Car
    const int nitems = 2; // number of struct fields
    int blocklengths[nitems] = { 1, 1 }; // lengths of types for each struct field
    MPI_Datatype types[nitems] = { MPI_INT, MPI_INT }; // MPI types of each struct field
    MPI_Datatype mpi_car_type; // the new MPI dataType
    MPI_Aint offsets[nitems]; // offset computed directly from fields

    offsets[0] = offsetof(Car, Model);
    offsets[1] = offsetof(Car, Color);

    // create the new dataType
    MPI_Type_create_struct(nitems, blocklengths, offsets, types, &mpi_car_type);
    MPI_Type_commit(&mpi_car_type); // commit operation

    Car car = { 0, 0 };
    
    if (rank == 0) 
    {
        car.Model = 4;
        car.Color = 100;

        MPI_Send(&car, 1, mpi_car_type, 1, tag, MPI_COMM_WORLD);

        std::cout<< "Process "<< rank<< ": sent structure car"<< std::endl;
    }
    else if (rank == 1) 
    {
        MPI_Status status;
        MPI_Recv(&car, 1, mpi_car_type, 0, tag, MPI_COMM_WORLD, &status);
        std::cout<< "Process "<< rank<< ": recv structure car"<< std::endl;
    }
    
    std::cout<< "Process "<< rank<< ": car.Model "<< car.Model<< " car.Color "<< car.Color<< std::endl;

    MPI_Type_free(&mpi_car_type); // destroy operation

    MPI_Finalize();
    return 0;
}

Overwriting main_struct.cpp


In [2]:
%%sh

# compile program
mkdir -p ./debug_struct
cd debug_struct
cmake -DSOURCES="main_struct.cpp" ..
make

-- Configuring done
-- Generating done
-- Build files have been written to: /home/geoscore/Desktop/GEO++/Courses/PhdMPI/2_MPI_Communications/debug_struct
Consolidate compiler generated dependencies of target 2_MPI_Communications
[ 50%] Building CXX object CMakeFiles/2_MPI_Communications.dir/main_struct.cpp.o
[100%] Linking CXX executable 2_MPI_Communications
[100%] Built target 2_MPI_Communications


In [3]:
%%sh

# run program
cd debug_struct
mpirun -np 3 2_MPI_Communications

Process 0: sent structure car
Process 0: car.Model 4 car.Color 100
Process 2: car.Model 0 car.Color 0
Process 1: recv structure car
Process 1: car.Model 4 car.Color 100
